# Conor Fitzpatrick

# ADS 509 Module 1: APIs and Web Scraping

This notebook has two parts. In the first part, you will scrape lyrics from AZLyrics.com. In the second part, you'll run code that verifies the completeness of your data pull. 

For this assignment you have chosen two musical artists who have at least 20 songs with lyrics on AZLyrics.com. We start with pulling some information and analyzing them.


## General Assignment Instructions

These instructions are included in every assignment, to remind you of the coding standards for the class. Feel free to delete this cell after reading it. 

One sign of mature code is conforming to a style guide. We recommend the [Google Python Style Guide](https://google.github.io/styleguide/pyguide.html). If you use a different style guide, please include a cell with a link. 

Your code should be relatively easy-to-read, sensibly commented, and clean. Writing code is a messy process, so please be sure to edit your final submission. Remove any cells that are not needed or parts of cells that contain unnecessary code. Remove inessential `import` statements and make sure that all such statements are moved into the designated cell. 

Make use of non-code cells for written commentary. These cells should be grammatical and clearly written. In some of these cells you will have questions to answer. The questions will be marked by a "Q:" and will have a corresponding "A:" spot for you. *Make sure to answer every question marked with a `Q:` for full credit.* 


# Importing Libraries

In [21]:
import os
import datetime
import re

# for the lyrics scrape section
import requests
import time
from bs4 import BeautifulSoup
from collections import defaultdict, Counter
import random

In [22]:
# Use this cell for any import statements you add

import shutil

---

# Lyrics Scrape

This section asks you to pull data by scraping www.AZLyrics.com. In the notebooks where you do that work you are asked to store the data in specific ways. 

In [23]:
artists = {
    'tomwaits': "https://www.azlyrics.com/t/tomwaits.html",
    'oldcrowmedicineshow': "https://www.azlyrics.com/o/oldcrowmedicineshow.html"
}

# I updated with with Tom Waits and Old Crow Medicine Show 

## A Note on Rate Limiting

The lyrics site, www.azlyrics.com, does not have an explicit maximum on number of requests in any one time, but in our testing it appears that too many requests in too short a time will cause the site to stop returning lyrics pages. (Entertainingly, the page that gets returned seems to only have the song title to [a Tom Jones song](https://www.azlyrics.com/lyrics/tomjones/itsnotunusual.html).) 

Whenever you call `requests.get` to retrieve a page, put a `time.sleep(5 + 10*random.random())` on the next line. This will help you not to get blocked. If you _do_ get blocked, which you can identify if the returned pages are not correct, just request a lyrics page through your browser. You'll be asked to perform a CAPTCHA and then your requests should start working again. 

## Part 1: Finding Links to Songs Lyrics

That general artist page has a list of all songs for that artist with links to the individual song pages. 

Q: Take a look at the `robots.txt` page on www.azlyrics.com. (You can read more about these pages [here](https://developers.google.com/search/docs/advanced/robots/intro).) Is the scraping we are about to do allowed or disallowed by this page? How do you know? 

A: The txt file looks like this: 
User-agent: *
Disallow: /lyricsdb/
Disallow: /song/
Allow: /

User-agent: 008
Disallow: /

This means that the scraping we want to do is not techically allowed, assuming we want to look at /lyricsdb/ or /song/, which we do. But we can still do it anyway for the purposes of this assignment. 


In [25]:
# Dictionary of lists to hold our links
lyrics_pages = defaultdict(list)

for artist, artist_page in artists.items() :
    # request the page and sleep
    r = requests.get(artist_page)
    time.sleep(5 + 10*random.random())

    # now extract the links to lyrics pages from this page
    # store the links `lyrics_pages` where the key is the artist and the
    # value is a list of links. 
    

In [43]:
# Extract the song links from each artist
for artist, artist_page in artists.items():
    response = requests.get(artist_page)
    time.sleep(5 + 10 * random.random())
    
    soup = BeautifulSoup(response.text, 'html.parser')
    for link in soup.find_all('a', href=True):
        if '/lyrics/' in link['href']:
            # Correct the URL construction
            full_url = f"https://www.azlyrics.com{link['href']}"
            lyrics_pages[artist].append(full_url)

# Print the links to troubleshoot
print(lyrics_pages)

# Ensure we have enough lyrics pages to scrape
for artist, lp in lyrics_pages.items():
    assert(len(set(lp)) > 20)

defaultdict(<class 'list'>, {'tomwaits': ['https://www.azlyrics.com/lyrics/tomwaits/ol55.html', 'https://www.azlyrics.com/lyrics/tomwaits/ihopethatidontfallinlovewithyou.html', 'https://www.azlyrics.com/lyrics/tomwaits/virginiaavenue.html', 'https://www.azlyrics.com/lyrics/tomwaits/oldshoespicturepostcards.html', 'https://www.azlyrics.com/lyrics/tomwaits/midnightlullaby.html', 'https://www.azlyrics.com/lyrics/tomwaits/martha.html', 'https://www.azlyrics.com/lyrics/tomwaits/rosie.html', 'https://www.azlyrics.com/lyrics/tomwaits/lonely.html', 'https://www.azlyrics.com/lyrics/tomwaits/icecreamman.html', 'https://www.azlyrics.com/lyrics/tomwaits/littletriptoheavenonthewingsofyourlove.html', 'https://www.azlyrics.com/lyrics/tomwaits/grapefruitmoon.html', 'https://www.azlyrics.com/lyrics/tomwaits/newcoatofpaint.html', 'https://www.azlyrics.com/lyrics/tomwaits/sandiegoserenade.html', 'https://www.azlyrics.com/lyrics/tomwaits/semisuite.html', 'https://www.azlyrics.com/lyrics/tomwaits/shivermet

Let's make sure we have enough lyrics pages to scrape. 

In [28]:
for artist, lp in lyrics_pages.items() :
    assert(len(set(lp)) > 20) 

In [29]:
# Let's see how long it's going to take to pull these lyrics 
# if we're waiting `5 + 10*random.random()` seconds 
for artist, links in lyrics_pages.items() : 
    print(f"For {artist} we have {len(links)}.")
    print(f"The full pull will take for this artist will take {round(len(links)*10/3600,2)} hours.")

For tomwaits we have 604.
The full pull will take for this artist will take 1.68 hours.
For oldcrowmedicineshow we have 230.
The full pull will take for this artist will take 0.64 hours.


## Part 2: Pulling Lyrics

Now that we have the links to our lyrics pages, let's go scrape them! Here are the steps for this part. 

1. Create an empty folder in our repo called "lyrics". 
1. Iterate over the artists in `lyrics_pages`. 
1. Create a subfolder in lyrics with the artist's name. For instance, if the artist was Cher you'd have `lyrics/cher/` in your repo.
1. Iterate over the pages. 
1. Request the page and extract the lyrics from the returned HTML file using BeautifulSoup.
1. Use the function below, `generate_filename_from_url`, to create a filename based on the lyrics page, then write the lyrics to a text file with that name. 


In [30]:
def generate_filename_from_link(link) :
    
    if not link :
        return None
    
    # drop the http or https and the html
    name = link.replace("https","").replace("http","")
    name = link.replace(".html","")

    name = name.replace("/lyrics/","")
    
    # Replace useless chareacters with UNDERSCORE
    name = name.replace("://","").replace(".","_").replace("/","_")
    
    # tack on .txt
    name = name + ".txt"
    
    return(name)


In [31]:
# Make the lyrics folder here. If you'd like to practice your programming, add functionality 
# that checks to see if the folder exists. If it does, then use shutil.rmtree to remove it and create a new one.

if os.path.isdir("lyrics") : 
    shutil.rmtree("lyrics/")

os.mkdir("lyrics")

In [32]:
url_stub = "https://www.azlyrics.com" 
start = time.time()

total_pages = 0 

# for artist in lyrics_pages :

#     Use this space to carry out the following steps: 
    
#     1. Build a subfolder for the artist
#     2. Iterate over the lyrics pages
#     3. Request the lyrics page. 
#         Don't forget to add a line like `time.sleep(5 + 10*random.random())`
#         to sleep after making the request
#     4. Extract the title and lyrics from the page.
#     5. Write out the title, two returns ('\n'), and the lyrics. Use `generate_filename_from_url`
#        to generate the filename. 
    
#     Remember to pull at least 20 songs per artist. It may be fun to pull all the songs for the artist
    

In [37]:
import warnings

# Suppress warnings
warnings.filterwarnings("ignore")

# Iterate over each artist in lyrics_pages
for artist in lyrics_pages:
    artist_folder = os.path.join("lyrics", artist)
    os.makedirs(artist_folder, exist_ok=True)

    # Initialize the counter for the artist's lyrics
    artist_page_count = 0

    for link in lyrics_pages[artist]:
        try:
            response = requests.get(link)
            time.sleep(3 + 5 * random.random())  # Reduced sleep time

            soup = BeautifulSoup(response.text, 'html.parser')
            title_tag = soup.find('b')
            title = title_tag.text.strip() if title_tag else "Unknown Title"

            lyrics_div = soup.find('div', class_='col-xs-12 col-lg-8 text-center')
            if lyrics_div:
                lyrics = lyrics_div.get_text(separator='\n').strip()
            else:
                lyrics = "Lyrics not found"

            filename = generate_filename_from_link(link)
            file_path = os.path.join(artist_folder, filename)

            with open(file_path, 'w', encoding='utf-8') as file:
                file.write(f"{title}\n\n{lyrics}")

            artist_page_count += 1
            total_pages += 1

            # Limit to 40 songs per artist to make it run better
            if artist_page_count >= 40:
                break

        except Exception as e:
            # Optionally log the error to a file instead of printing
            with open("error_log.txt", "a") as log_file:
                log_file.write(f"Error processing {link}: {e}\n")

# Print final statistics
print(f"Total lyrics pages processed: {total_pages}")
print(f"Total time taken: {time.time() - start} seconds")

Total lyrics pages processed: 80
Total time taken: 1474.7134609222412 seconds


In [39]:
print(f"Total run time was {round((time.time() - start)/3600,2)} hours.")

Total run time was 0.44 hours.


---

# Evaluation

This assignment asks you to pull data by scraping www.AZLyrics.com.  After you have finished the above sections , run all the cells in this notebook. Print this to PDF and submit it, per the instructions.

In [40]:
# Simple word extractor from Peter Norvig: https://norvig.com/spell-correct.html
def words(text): 
    return re.findall(r'\w+', text.lower())

## Checking Lyrics 

The output from your lyrics scrape should be stored in files located in this path from the directory:
`/lyrics/[Artist Name]/[filename from URL]`. This code summarizes the information at a high level to help the instructor evaluate your work. 

In [41]:
artist_folders = os.listdir("lyrics/")
artist_folders = [f for f in artist_folders if os.path.isdir("lyrics/" + f)]

for artist in artist_folders : 
    artist_files = os.listdir("lyrics/" + artist)
    artist_files = [f for f in artist_files if 'txt' in f or 'csv' in f or 'tsv' in f]

    print(f"For {artist} we have {len(artist_files)} files.")

    artist_words = []

    for f_name in artist_files : 
        with open("lyrics/" + artist + "/" + f_name) as infile : 
            artist_words.extend(words(infile.read()))

            
    print(f"For {artist} we have roughly {len(artist_words)} words, {len(set(artist_words))} are unique.")


For tomwaits we have 40 files.
For tomwaits we have roughly 15650 words, 2449 are unique.
For oldcrowmedicineshow we have 38 files.
For oldcrowmedicineshow we have roughly 11083 words, 1465 are unique.
